In [1]:
import json
import numpy as np
import scipy as sp
#matplotlib inline
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import random
import bisect
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds, eigs
from numpy.linalg import matrix_rank
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
import pandas as pd
# Import recommenders
import content_recommender
import collaborative_filtering_item as cf_item
import collaborative_filtering_user as cf_user
import latent_factor_model
import svd4rec
import clustering

# Open JSON file
restaurant_data = open('yelp_dataset/TX_restaurants.json')
restaurants = json.load(restaurant_data)
print('Total number of restaurants in dataset: ', len(restaurants))

user_data = open('yelp_dataset/TX_users.json')
users = json.load(user_data)
print('Total number of users in dataset: ', len(users))

review_data = open('yelp_dataset/TX_reviews.json')
reviews = json.load(review_data)
print('Total number of reviews in dataset: ', len(reviews))

kevan_data = open('yelp_dataset/kevan.json')
kevan = json.load(kevan_data)

anti_social_data = open('yelp_dataset/anti_social.json')
anti_social = json.load(anti_social_data)

# build user and restaurant lists
user_list = []
for u in users:
    user_list.append(u['user_id'])
rest_list = []
for r in restaurants:
    rest_list.append(r['business_id'])

Total number of restaurants in dataset:  3270
Total number of users in dataset:  798
Total number of reviews in dataset:  62936


# CF User-user

In [2]:
#KEVAN
user_user_input = "-OGWTHZng0QNhvc8dhIjyQ"


In [3]:
# Load the data
user_ids = []
business_ids = []
stars = []

for x in range(0, len(reviews)):
    user_ids.append(reviews[x]['user_id'])
    business_ids.append((reviews[x]['business_id']))
    stars.append(np.float64((reviews[x]['stars'])))

#if user_input not in user_ids:
#    print(user_input, " is not a valid id")
#    quit()

In [4]:
# build utility matrix
user_matrix_dict = {"user_id": user_ids, 'business_id': business_ids, 'stars': stars}
user_matrix_df = pd.DataFrame(user_matrix_dict, columns=['user_id', 'business_id', 'stars'])
user_matrix_df

,user_id,business_id,stars
0,wJHy7ZJG_EvLFQDRms5rXQ,75HV-KqCtn_oHeiLiGlO_w,4.0
1,HHEXgBRDdkFSiDu1gDSdKg,mrABhBpFvXTFrLiLiz286g,4.0
2,QdhsBITt2VqQzwNxRu4QjQ,dQ80ktWSRj5UIk96bRUmWw,4.0
3,7jc9f2Nn2S--5b-G5cfJ-A,_jYt69Zx1SUo_V9z0m7Ljg,4.0
4,8dXZLf8hM4JDIU0Rk3X_Tg,IBK8opCBFSoZCSdURwBhJQ,2.0
...,...,...,...
62931,QcVB_54m6o8jbq5PJBFsHQ,2vppZx0rTDZtCzw-NljdRQ,5.0
62932,DYONpUFtHQK_cYD9ylG_4Q,gBFtNT6fh8C-7Clo-aDLvA,3.0
62933,1-c5fkvmpp6RqOLlJYd98A,AWGRRRSaxAHI1zeWYb06sg,4.0
62934,2uV7zGYSqTXb-sDCeiZ1NA,zM98ZSIJyuBQabyYornLpw,4.0


In [5]:
user_pivot_matrix = user_matrix_df.pivot_table(index=['user_id'], columns=['business_id'], values=['stars'])
user_pivot_matrix

stars                         \
business_id            -4ofMtrD7pSpZIX5pnDkig -85pDrVcAdzNK55bFpintg   
user_id                                                                
-0MQ4webH2uc1ZAsGsNENg                    NaN                    NaN   
-2sNTzGyci98Mp9PmPRg8w                    NaN                    NaN   
-4RH83ibNRpwj8NlBwtMdQ                    NaN                    NaN   
-8QoOIfvwwxJ4sY201WP5A                    NaN                    NaN   
-OGWTHZng0QNhvc8dhIjyQ                    NaN                    NaN   
...                                       ...                    ...   
zgLUWGBKrSoIIH3E2XLvNQ                    NaN                    NaN   
zlL_PbFTXjrGXhTbPsjmlQ                    NaN                    NaN   
zonT-KQFJiejFNUrs2KR0Q                    NaN                    NaN   
zqMc0O7vDj82Sz5Fler1EA                    NaN                    NaN   
zx8hZcEiqwLnPJWSsblyzA                    4.0                    NaN   

                                                                      \
business_id            -BBSLCjzw3i2PHuwJ_dabA -Ce8p148xb0-4dv_3zwm3A   
user_id                                                                
-0MQ4webH2uc1ZAsGsNENg                    NaN                    NaN   
-2sNTzGyci98Mp9PmPRg8w                    NaN                    NaN   
-4RH83ibNRpwj8NlBwtMdQ                    NaN                    NaN   
-8QoOIfvwwxJ4sY201WP5A                    NaN                    NaN   
-OGWTHZng0QNhvc8dhIjyQ                    NaN                    NaN   
...                                       ...                    ...   
zgLUWGBKrSoIIH3E2XLvNQ                    2.0                    NaN   
zlL_PbFTXjrGXhTbPsjmlQ                    NaN                    NaN   
zonT-KQFJiejFNUrs2KR0Q                    NaN                    NaN   
zqMc0O7vDj82Sz5Fler1EA                    NaN                    5.0   
zx8hZcEiqwLnPJWSsblyzA                    NaN                    NaN   

                                                                      \
business_id            -WA4tNsXZq5sxcGw8a5_IQ -_GnwXmzC3DXsHR9nyaC2g   
user_id                                                                
-0MQ4webH2uc1ZAsGsNENg                    NaN                    1.0   
-2sNTzGyci98Mp9PmPRg8w                    NaN                    NaN   
-4RH83ibNRpwj8NlBwtMdQ                    NaN                    NaN   
-8QoOIfvwwxJ4sY201WP5A                    NaN                    NaN   
-OGWTHZng0QNhvc8dhIjyQ                    NaN                    NaN   
...                                       ...                    ...   
zgLUWGBKrSoIIH3E2XLvNQ                    NaN                    NaN   
zlL_PbFTXjrGXhTbPsjmlQ                    3.5                    NaN   
zonT-KQFJiejFNUrs2KR0Q                    NaN                    2.0   
zqMc0O7vDj82Sz5Fler1EA                    NaN                    NaN   
zx8hZcEiqwLnPJWSsblyzA                    NaN                    NaN   

                                                                      \
business_id            -jQOsyXnAMKu6ND7ongXFg -qjkQHH-5O8BAztc6udOuw   
user_id                                                                
-0MQ4webH2uc1ZAsGsNENg                    NaN                    NaN   
-2sNTzGyci98Mp9PmPRg8w                    NaN                    NaN   
-4RH83ibNRpwj8NlBwtMdQ                    NaN                    NaN   
-8QoOIfvwwxJ4sY201WP5A                    NaN                    NaN   
-OGWTHZng0QNhvc8dhIjyQ                    NaN                    NaN   
...                                       ...                    ...   
zgLUWGBKrSoIIH3E2XLvNQ                    NaN                    NaN   
zlL_PbFTXjrGXhTbPsjmlQ                    NaN                    NaN   
zonT-KQFJiejFNUrs2KR0Q                    NaN                    NaN   
zqMc0O7vDj82Sz5Fler1EA                    NaN                    5.0   
zx8hZcEiqwLnPJWSsblyzA                    NaN                    NaN   

    

In [6]:
# normalize matrix
user_pivot_matrix_ = user_pivot_matrix.sub(user_pivot_matrix.mean(axis=1), axis=0)
user_pivot_matrix_norm = user_pivot_matrix_.replace(np.NaN, 0)
user_pivot_matrix_norm

stars                         \
business_id            -4ofMtrD7pSpZIX5pnDkig -85pDrVcAdzNK55bFpintg   
user_id                                                                
-0MQ4webH2uc1ZAsGsNENg               0.000000                    0.0   
-2sNTzGyci98Mp9PmPRg8w               0.000000                    0.0   
-4RH83ibNRpwj8NlBwtMdQ               0.000000                    0.0   
-8QoOIfvwwxJ4sY201WP5A               0.000000                    0.0   
-OGWTHZng0QNhvc8dhIjyQ               0.000000                    0.0   
...                                       ...                    ...   
zgLUWGBKrSoIIH3E2XLvNQ               0.000000                    0.0   
zlL_PbFTXjrGXhTbPsjmlQ               0.000000                    0.0   
zonT-KQFJiejFNUrs2KR0Q               0.000000                    0.0   
zqMc0O7vDj82Sz5Fler1EA               0.000000                    0.0   
zx8hZcEiqwLnPJWSsblyzA               0.462264                    0.0   

                                                                      \
business_id            -BBSLCjzw3i2PHuwJ_dabA -Ce8p148xb0-4dv_3zwm3A   
user_id                                                                
-0MQ4webH2uc1ZAsGsNENg                0.00000               0.000000   
-2sNTzGyci98Mp9PmPRg8w                0.00000               0.000000   
-4RH83ibNRpwj8NlBwtMdQ                0.00000               0.000000   
-8QoOIfvwwxJ4sY201WP5A                0.00000               0.000000   
-OGWTHZng0QNhvc8dhIjyQ                0.00000               0.000000   
...                                       ...                    ...   
zgLUWGBKrSoIIH3E2XLvNQ               -1.45935               0.000000   
zlL_PbFTXjrGXhTbPsjmlQ                0.00000               0.000000   
zonT-KQFJiejFNUrs2KR0Q                0.00000               0.000000   
zqMc0O7vDj82Sz5Fler1EA                0.00000               0.955556   
zx8hZcEiqwLnPJWSsblyzA                0.00000               0.000000   

                                                                      \
business_id            -WA4tNsXZq5sxcGw8a5_IQ -_GnwXmzC3DXsHR9nyaC2g   
user_id                                                                
-0MQ4webH2uc1ZAsGsNENg               0.000000              -2.695122   
-2sNTzGyci98Mp9PmPRg8w               0.000000               0.000000   
-4RH83ibNRpwj8NlBwtMdQ               0.000000               0.000000   
-8QoOIfvwwxJ4sY201WP5A               0.000000               0.000000   
-OGWTHZng0QNhvc8dhIjyQ               0.000000               0.000000   
...                                       ...                    ...   
zgLUWGBKrSoIIH3E2XLvNQ               0.000000               0.000000   
zlL_PbFTXjrGXhTbPsjmlQ              -0.977564               0.000000   
zonT-KQFJiejFNUrs2KR0Q               0.000000              -1.414773   
zqMc0O7vDj82Sz5Fler1EA               0.000000               0.000000   
zx8hZcEiqwLnPJWSsblyzA               0.000000               0.000000   

                                                                      \
business_id            -jQOsyXnAMKu6ND7ongXFg -qjkQHH-5O8BAztc6udOuw   
user_id                                                                
-0MQ4webH2uc1ZAsGsNENg                    0.0               0.000000   
-2sNTzGyci98Mp9PmPRg8w                    0.0               0.000000   
-4RH83ibNRpwj8NlBwtMdQ                    0.0               0.000000   
-8QoOIfvwwxJ4sY201WP5A                    0.0               0.000000   
-OGWTHZng0QNhvc8dhIjyQ                    0.0               0.000000   
...                                       ...                    ...   
zgLUWGBKrSoIIH3E2XLvNQ                    0.0               0.000000   
zlL_PbFTXjrGXhTbPsjmlQ                    0.0               0.000000   
zonT-KQFJiejFNUrs2KR0Q                    0.0               0.000000   
zqMc0O7vDj82Sz5Fler1EA                    0.0               0.955556   
zx8hZcEiqwLnPJWSsblyzA                    0.0               0.000000   

    

In [7]:
# Get Pearson Correlation
user_pivot_matrix_norm_corr = user_pivot_matrix_norm.T.corr(method='pearson')
user_pivot_matrix_norm_corr

user_id,-0MQ4webH2uc1ZAsGsNENg,-2sNTzGyci98Mp9PmPRg8w,-4RH83ibNRpwj8NlBwtMdQ,-8QoOIfvwwxJ4sY201WP5A,-OGWTHZng0QNhvc8dhIjyQ,-UMgfkqsTRc6JQxU-G1nXQ,-_1hQemvkJQE44pKnU_XQg,-_tiDCgVStFg9hpwHVR2Lg,-dMuB2gJ2z3wdatazYNu4g,-hKniZN2OdshWLHYuj21jQ,...,zTn90jWKmFW65x5U97bCVQ,zWGj3oOA_X8QPho_l5BRKg,z_0fWp3wKKdEruMABeX7WQ,zcMGNt3hwNABJSXiPOEHqg,zgDADFTL6rSi6ELUMJPuxw,zgLUWGBKrSoIIH3E2XLvNQ,zlL_PbFTXjrGXhTbPsjmlQ,zonT-KQFJiejFNUrs2KR0Q,zqMc0O7vDj82Sz5Fler1EA,zx8hZcEiqwLnPJWSsblyzA
user_id,,,,,,,,,,,,,,,,,,,,,
-0MQ4webH2uc1ZAsGsNENg,1.000000,0.049484,0.000695,-0.017389,0.021323,0.025918,-2.430589e-02,0.000958,0.062182,0.031411,...,5.240809e-02,-0.023538,0.004502,-0.029309,0.023176,0.057443,0.016859,0.088027,0.009431,0.004551
-2sNTzGyci98Mp9PmPRg8w,0.049484,1.000000,-0.061872,-0.044385,-0.057048,0.062254,-2.107242e-03,-0.059166,0.041352,-0.040082,...,-1.152317e-02,0.043950,-0.018734,0.043131,0.004525,0.073277,0.049402,0.097794,0.061602,0.007728
-4RH83ibNRpwj8NlBwtMdQ,0.000695,-0.061872,1.000000,-0.016189,0.007253,0.006630,1.548356e-01,-0.065432,-0.002634,-0.037740,...,7.830044e-33,0.007154,0.179850,0.028981,-0.101078,-0.027943,-0.000109,-0.023195,0.069589,0.049431
-8QoOIfvwwxJ4sY201WP5A,-0.017389,-0.044385,-0.016189,1.000000,0.023978,-0.000936,6.200968e-03,0.122690,-0.006173,0.025263,...,-3.717459e-02,-0.032062,-0.026172,-0.003184,0.057426,0.041903,-0.028509,0.035222,0.023214,-0.052345
-OGWTHZng0QNhvc8dhIjyQ,0.021323,-0.057048,0.007253,0.023978,1.000000,0.051352,6.985569e-02,-0.057261,-0.008895,-0.003656,...,2.166851e-03,-0.065427,0.031008,-0.002212,-0.022565,0.038208,-0.018598,0.022246,0.062019,-0.031041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zgLUWGBKrSoIIH3E2XLvNQ,0.057443,0.073277,-0.027943,0.041903,0.038208,0.039724,-3.208543e-33,0.129420,0.022842,-0.012654,...,1.792396e-32,-0.041031,0.013238,-0.002031,-0.006078,1.000000,0.030474,0.036875,-0.049450,-0.075666
zlL_PbFTXjrGXhTbPsjmlQ,0.016859,0.049402,-0.000109,-0.028509,-0.018598,-0.012928,-2.769274e-03,0.061128,0.066221,0.074245,...,-1.120185e-02,0.066788,-0.000768,0.001499,0.012969,0.030474,1.000000,0.012916,-0.034659,0.061204
zonT-KQFJiejFNUrs2KR0Q,0.088027,0.097794,-0.023195,0.035222,0.022246,0.035972,9.220066e-03,-0.011101,0.011581,0.004925,...,5.163277e-02,-0.039390,-0.003121,-0.038637,0.004855,0.036875,0.012916,1.000000,0.062322,-0.039627


In [8]:
# Remove diagonal 1's in Pearson Correlation
user_pivot_matrix_norm_corr.values[[np.arange(user_pivot_matrix_norm_corr.shape[0])] * 2] = 0
user_pivot_matrix_norm_corr

<ipython-input-8-701b30b8ad43>:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  user_pivot_matrix_norm_corr.values[[np.arange(user_pivot_matrix_norm_corr.shape[0])] * 2] = 0


user_id,-0MQ4webH2uc1ZAsGsNENg,-2sNTzGyci98Mp9PmPRg8w,-4RH83ibNRpwj8NlBwtMdQ,-8QoOIfvwwxJ4sY201WP5A,-OGWTHZng0QNhvc8dhIjyQ,-UMgfkqsTRc6JQxU-G1nXQ,-_1hQemvkJQE44pKnU_XQg,-_tiDCgVStFg9hpwHVR2Lg,-dMuB2gJ2z3wdatazYNu4g,-hKniZN2OdshWLHYuj21jQ,...,zTn90jWKmFW65x5U97bCVQ,zWGj3oOA_X8QPho_l5BRKg,z_0fWp3wKKdEruMABeX7WQ,zcMGNt3hwNABJSXiPOEHqg,zgDADFTL6rSi6ELUMJPuxw,zgLUWGBKrSoIIH3E2XLvNQ,zlL_PbFTXjrGXhTbPsjmlQ,zonT-KQFJiejFNUrs2KR0Q,zqMc0O7vDj82Sz5Fler1EA,zx8hZcEiqwLnPJWSsblyzA
user_id,,,,,,,,,,,,,,,,,,,,,
-0MQ4webH2uc1ZAsGsNENg,0.000000,0.049484,0.000695,-0.017389,0.021323,0.025918,-2.430589e-02,0.000958,0.062182,0.031411,...,5.240809e-02,-0.023538,0.004502,-0.029309,0.023176,0.057443,0.016859,0.088027,0.009431,0.004551
-2sNTzGyci98Mp9PmPRg8w,0.049484,0.000000,-0.061872,-0.044385,-0.057048,0.062254,-2.107242e-03,-0.059166,0.041352,-0.040082,...,-1.152317e-02,0.043950,-0.018734,0.043131,0.004525,0.073277,0.049402,0.097794,0.061602,0.007728
-4RH83ibNRpwj8NlBwtMdQ,0.000695,-0.061872,0.000000,-0.016189,0.007253,0.006630,1.548356e-01,-0.065432,-0.002634,-0.037740,...,7.830044e-33,0.007154,0.179850,0.028981,-0.101078,-0.027943,-0.000109,-0.023195,0.069589,0.049431
-8QoOIfvwwxJ4sY201WP5A,-0.017389,-0.044385,-0.016189,0.000000,0.023978,-0.000936,6.200968e-03,0.122690,-0.006173,0.025263,...,-3.717459e-02,-0.032062,-0.026172,-0.003184,0.057426,0.041903,-0.028509,0.035222,0.023214,-0.052345
-OGWTHZng0QNhvc8dhIjyQ,0.021323,-0.057048,0.007253,0.023978,0.000000,0.051352,6.985569e-02,-0.057261,-0.008895,-0.003656,...,2.166851e-03,-0.065427,0.031008,-0.002212,-0.022565,0.038208,-0.018598,0.022246,0.062019,-0.031041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zgLUWGBKrSoIIH3E2XLvNQ,0.057443,0.073277,-0.027943,0.041903,0.038208,0.039724,-3.208543e-33,0.129420,0.022842,-0.012654,...,1.792396e-32,-0.041031,0.013238,-0.002031,-0.006078,0.000000,0.030474,0.036875,-0.049450,-0.075666
zlL_PbFTXjrGXhTbPsjmlQ,0.016859,0.049402,-0.000109,-0.028509,-0.018598,-0.012928,-2.769274e-03,0.061128,0.066221,0.074245,...,-1.120185e-02,0.066788,-0.000768,0.001499,0.012969,0.030474,0.000000,0.012916,-0.034659,0.061204
zonT-KQFJiejFNUrs2KR0Q,0.088027,0.097794,-0.023195,0.035222,0.022246,0.035972,9.220066e-03,-0.011101,0.011581,0.004925,...,5.163277e-02,-0.039390,-0.003121,-0.038637,0.004855,0.036875,0.012916,0.000000,0.062322,-0.039627


In [9]:
# k Nearest Neighbors
k = 20
user_top_matrix = user_pivot_matrix_norm_corr.sort_values(by=[user_user_input], ascending=False)
user_top = user_top_matrix[user_user_input]

user_top_users = user_top.index.values[1:k+1]
user_top_correlations = user_top.values[1:k+1]

# Top Results
print("Recommendation for other users similar to user: ", user_user_input)

#for i in range(len(user_top_users)):
#    print('{0}: {1}, with similarity of {2}'.format(i, user_top_users[i], user_top_correlations[i]))
    
users_name=[]
for x in range(len(users)):
    for y in range(len(user_top_users)):
        if (users[x]['user_id']==user_top_users[y]):
            users_name.append(users[x]["name"])
 
    
user_results_df = pd.DataFrame(data=[user_top_users, users_name, user_top_correlations]).T
user_results_df.columns=[ "user_id","Name", "Similarity"]
#print(user_results_df)    
user_results_df

Recommendation for other users similar to user:  -OGWTHZng0QNhvc8dhIjyQ


,user_id,Name,Similarity
0,VRYhpVtbrQhC0BFhfOpU5w,Liz,0.142309
1,Vbtm8ckaQ8WROtnJcS0OwQ,Jana,0.128816
2,rIx1lD7qCoYbdKtcTUF7DA,Nicolette,0.124545
3,HMUnp55Q8_vxIEPl1V0W_g,Tee,0.123292
4,bGzQELylOfNPi_XpitxPTQ,Don,0.119193
5,TpEUaUEu-7y3_9Wkvly8FA,Cyrus,0.10942
6,lbvIuUk282-I3Q_2I9_YrQ,Nicole,0.108633
7,YZuqPFBcGPSnucrxgq-RZQ,Trista,0.107092
8,dusl6MaLSfYbE83_iVO2KQ,Shinta,0.104464
9,cNlHuQWjknu7LsjFV1c47A,Meg,0.103167


In [10]:
item_user_input = "hpAOH8cmY5WHjnl8jGQ1dA"

In [11]:
for x in range(len(reviews)):
    for y in range(len(user_top_users)):
        if bool(reviews[x]['user_id']==user_top_users[y]) & bool(reviews[x]['business_id']==item_user_input):
            print(reviews[x]['user_id'])

UlXJP4fzVg-mLSluUnY5rw
BV9925KFD6KWxPpkSNN0dQ
tmpfESJAXQXqyvJFEl1Acg


In [12]:
percent_na = (user_pivot_matrix.isnull().sum().sum())/ (798*779)
percent_na 

0.907197390137732

# CF User-user, Predicted Ratings


In [13]:
item_matrix_dict = {"user_id": user_ids, 'business_id': business_ids, 'stars': stars}
item_matrix_df = pd.DataFrame(item_matrix_dict, columns=['user_id', 'business_id', 'stars'])
item_pivot_matrix = item_matrix_df.pivot_table(index=['business_id'], columns=['user_id'], values=['stars'])

item_pivot_matrix_means=item_pivot_matrix.mean(axis=1)
user_pivot_matrix_means = user_pivot_matrix.mean(axis=1)
user_pivot_matrix_entiremean = np.nanmean(user_pivot_matrix)

#print("Average Rating of 1886: ", item_pivot_matrix_means[item_user_input])
#print("Average Rating of Kevan:", user_pivot_matrix_means[user_user_input])
#print("Average of all Users:", user_pivot_matrix_entiremean)

pred_rating=0
sim_numer=[]
sim_denom=[]
mu = user_pivot_matrix_entiremean
b_x = item_pivot_matrix_means[item_user_input] - mu 
b_i = user_pivot_matrix_means[user_user_input]  -  mu
b_xi = mu + b_x + b_i

#Calculate predicted rating for each recommended restaurant
for x in range(len(user_top_users)): 
    for y in range(len(reviews)):
        
        #checking to see if there are other restaurants reviewed by same user 
        #if it is, then calculate predicted rating
        if bool(reviews[y]['user_id']==user_top_users[x]) & bool(reviews[y]['business_id']==item_user_input):
            sim_numer.append(user_top_correlations[x]*(reviews[y]['stars'] - (mu + b_x + user_pivot_matrix_means[user_top_users[x]]-mu)))
            sim_denom.append(user_top_correlations[x])
            pred_rating = b_xi + (sum(sim_numer)/sum(sim_denom))



print("Predicted Rating for Restaurant: ", item_user_input,"is ", pred_rating)

Predicted Rating for Restaurant:  hpAOH8cmY5WHjnl8jGQ1dA is  3.584160079234785


# CF Item-item

In [14]:
item_user_input = "hpAOH8cmY5WHjnl8jGQ1dA"

In [15]:
# build utility matrix
item_matrix_dict = {"user_id": user_ids, 'business_id': business_ids, 'stars': stars}
item_matrix_df = pd.DataFrame(item_matrix_dict, columns=['user_id', 'business_id', 'stars'])

item_pivot_matrix = item_matrix_df.pivot_table(index=['business_id'], columns=['user_id'], values=['stars'])
item_pivot_matrix



stars                         \
user_id                -0MQ4webH2uc1ZAsGsNENg -2sNTzGyci98Mp9PmPRg8w   
business_id                                                            
-4ofMtrD7pSpZIX5pnDkig                    NaN                    NaN   
-85pDrVcAdzNK55bFpintg                    NaN                    NaN   
-BBSLCjzw3i2PHuwJ_dabA                    NaN                    NaN   
-Ce8p148xb0-4dv_3zwm3A                    NaN                    NaN   
-WA4tNsXZq5sxcGw8a5_IQ                    NaN                    NaN   
...                                       ...                    ...   
zM98ZSIJyuBQabyYornLpw                    NaN                    NaN   
zVILoCmmDSgcM5bc2BKVWg                    NaN                    NaN   
zZIWHmITwqaC8OhnPkjEIQ                    NaN                    NaN   
zerPYZ-lid5CG27a1-7YMw                    NaN                    NaN   
zyHMtStYlKG67WRprp6GZQ                    NaN                    3.0   

                                                                      \
user_id                -4RH83ibNRpwj8NlBwtMdQ -8QoOIfvwwxJ4sY201WP5A   
business_id                                                            
-4ofMtrD7pSpZIX5pnDkig                    NaN                    NaN   
-85pDrVcAdzNK55bFpintg                    NaN                    NaN   
-BBSLCjzw3i2PHuwJ_dabA                    NaN                    NaN   
-Ce8p148xb0-4dv_3zwm3A                    NaN                    NaN   
-WA4tNsXZq5sxcGw8a5_IQ                    NaN                    NaN   
...                                       ...                    ...   
zM98ZSIJyuBQabyYornLpw                    NaN                    NaN   
zVILoCmmDSgcM5bc2BKVWg                    NaN                    NaN   
zZIWHmITwqaC8OhnPkjEIQ                    NaN                    NaN   
zerPYZ-lid5CG27a1-7YMw                    NaN                    NaN   
zyHMtStYlKG67WRprp6GZQ                    NaN                    NaN   

                                                                      \
user_id                -OGWTHZng0QNhvc8dhIjyQ -UMgfkqsTRc6JQxU-G1nXQ   
business_id                                                            
-4ofMtrD7pSpZIX5pnDkig                    NaN                    NaN   
-85pDrVcAdzNK55bFpintg                    NaN                    NaN   
-BBSLCjzw3i2PHuwJ_dabA                    NaN                    NaN   
-Ce8p148xb0-4dv_3zwm3A                    NaN                    NaN   
-WA4tNsXZq5sxcGw8a5_IQ                    NaN                    NaN   
...                                       ...                    ...   
zM98ZSIJyuBQabyYornLpw                    NaN                    NaN   
zVILoCmmDSgcM5bc2BKVWg                    NaN                    NaN   
zZIWHmITwqaC8OhnPkjEIQ                    NaN                    NaN   
zerPYZ-lid5CG27a1-7YMw                    NaN                    NaN   
zyHMtStYlKG67WRprp6GZQ                    NaN                    NaN   

                                                                      \
user_id                -_1hQemvkJQE44pKnU_XQg -_tiDCgVStFg9hpwHVR2Lg   
business_id                                                            
-4ofMtrD7pSpZIX5pnDkig                    NaN                    NaN   
-85pDrVcAdzNK55bFpintg                    NaN                    NaN   
-BBSLCjzw3i2PHuwJ_dabA                    NaN                    NaN   
-Ce8p148xb0-4dv_3zwm3A                    NaN                    NaN   
-WA4tNsXZq5sxcGw8a5_IQ                    NaN                    NaN   
...                                       ...                    ...   
zM98ZSIJyuBQabyYornLpw                    NaN                    NaN   
zVILoCmmDSgcM5bc2BKVWg                    NaN                    NaN   
zZIWHmITwqaC8OhnPkjEIQ                    NaN                    NaN   
zerPYZ-lid5CG27a1-7YMw                    NaN                    NaN   
zyHMtStYlKG67WRprp6GZQ                    NaN                    NaN   

    

In [16]:
# normalize matrix
item_pivot_matrix_ = item_pivot_matrix.sub(item_pivot_matrix.mean(axis=1), axis=0)
item_pivot_matrix_norm = item_pivot_matrix_.replace(np.NaN, 0)

# Get Pearson Correlation
item_pivot_matrix_norm_corr = item_pivot_matrix_norm.T.corr(method='pearson')
item_pivot_matrix_norm_corr

business_id,-4ofMtrD7pSpZIX5pnDkig,-85pDrVcAdzNK55bFpintg,-BBSLCjzw3i2PHuwJ_dabA,-Ce8p148xb0-4dv_3zwm3A,-WA4tNsXZq5sxcGw8a5_IQ,-_GnwXmzC3DXsHR9nyaC2g,-jQOsyXnAMKu6ND7ongXFg,-qjkQHH-5O8BAztc6udOuw,-waa6lH9SmAkaKiLx74_FA,0-Y_m4TJZufHBVozbMOqbA,...,z3JW-nsdfC8ijst_PSeMIA,z4fQL2IHUR4ENonH7A_9tQ,zDVjN3dC9EfQno21m3O8Pg,zE0EGIsOdAO83xE3CypN-A,zFaHweOJ40jjtvpGTjlspw,zM98ZSIJyuBQabyYornLpw,zVILoCmmDSgcM5bc2BKVWg,zZIWHmITwqaC8OhnPkjEIQ,zerPYZ-lid5CG27a1-7YMw,zyHMtStYlKG67WRprp6GZQ
business_id,,,,,,,,,,,,,,,,,,,,,
-4ofMtrD7pSpZIX5pnDkig,1.000000,0.056411,-0.072335,0.041063,0.060966,-0.007827,2.512509e-02,-0.046471,0.040338,3.393558e-02,...,-0.012263,-0.009431,0.041278,0.017189,0.068412,0.006842,0.059086,0.019013,0.054048,-0.011086
-85pDrVcAdzNK55bFpintg,0.056411,1.000000,0.046330,-0.000210,0.187103,0.050058,-5.549443e-02,-0.058046,0.026660,1.004538e-02,...,0.074965,-0.020501,0.002276,0.060242,-0.017708,0.029968,0.031759,-0.014843,-0.055632,0.108989
-BBSLCjzw3i2PHuwJ_dabA,-0.072335,0.046330,1.000000,0.025036,0.068375,-0.005863,-2.799155e-35,0.002657,0.031660,9.713662e-04,...,-0.001583,0.039443,-0.002946,0.007719,-0.045065,0.024281,0.022258,-0.017764,0.036193,0.066448
-Ce8p148xb0-4dv_3zwm3A,0.041063,-0.000210,0.025036,1.000000,0.010136,0.004092,0.000000e+00,0.028113,0.015312,2.736732e-02,...,0.011726,0.011726,0.059152,-0.061176,0.028604,0.008247,0.007787,0.044148,-0.034957,0.126770
-WA4tNsXZq5sxcGw8a5_IQ,0.060966,0.187103,0.068375,0.010136,1.000000,-0.034675,-1.300332e-02,-0.020354,0.044614,-1.401190e-02,...,-0.036832,0.030202,0.014640,0.065010,-0.007876,0.011375,0.012659,0.027582,-0.023224,0.050500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zM98ZSIJyuBQabyYornLpw,0.006842,0.029968,0.024281,0.008247,0.011375,-0.015822,4.867189e-02,-0.021857,0.010357,-6.720875e-33,...,0.026298,-0.025788,0.074951,0.038637,0.010219,1.000000,0.051849,0.045119,0.002268,0.009772
zVILoCmmDSgcM5bc2BKVWg,0.059086,0.031759,0.022258,0.007787,0.012659,0.017890,6.243867e-02,-0.049062,0.076542,-2.892728e-02,...,-0.015580,-0.014799,0.041783,-0.048209,0.042871,0.051849,1.000000,0.009565,-0.017223,0.002137
zZIWHmITwqaC8OhnPkjEIQ,0.019013,-0.014843,-0.017764,0.044148,0.027582,0.006815,5.479551e-02,-0.005566,0.044012,-2.025088e-02,...,-0.001296,0.124257,-0.040802,0.081895,0.098298,0.045119,0.009565,1.000000,0.008010,-0.027544


In [17]:
# k Nearest Neighbors
k = 20
item_top_matrix = item_pivot_matrix_norm_corr.sort_values(by=[item_user_input], ascending=False)
item_top = item_top_matrix[item_user_input]

item_top_businesses = item_top.index.values[1:k+1]
item_top_correlations = item_top.values[1:k+1]
# Top Results
print("Recommendation for other restaurants similar to restaurant: ", item_user_input)

restaurant_name=[]
for x in range(len(restaurants)):
    for y in range(len(item_top_businesses)):
        if (restaurants[x]['business_id']==item_top_businesses[y]):
            restaurant_name.append(restaurants[x]['name'])
 
    
item_results_df = pd.DataFrame(data=[item_top_businesses, restaurant_name, item_top_correlations]).T
item_results_df.columns=[ "business_id","Restaurant", "Similarity"]
print(item_results_df)

Recommendation for other restaurants similar to restaurant:  hpAOH8cmY5WHjnl8jGQ1dA
               business_id                             Restaurant Similarity
0   jGm7oBtZCsc8BIWBnq4i3A               Waltons Fancy and Staple   0.272285
1   r3ZqJqJo6oCV3I2D4SYYkQ                    Cabo Bob's Burritos   0.240397
2   PceDd3uYY5VNBeI_zB2RBA             Quack's 43rd Street Bakery   0.201356
3   iyeJrTWE3_Xtup77dC_Mtg                          Newk's Eatery    0.18088
4   z4fQL2IHUR4ENonH7A_9tQ     Rudy's "Country Store" and Bar-B-Q   0.169892
5   J7Hs5_PGlZajNRxGptGRvw                   Lucy's Fried Chicken   0.159573
6   M-kdNZD-RhloIH5NZtS59A                       Home Slice Pizza   0.149678
7   xoBBLN7uDy_oI4PiKD4VWA                  Sugar Mama's Bakeshop   0.148285
8   QF9-_EoCi2bVCBjLhd4gtg                  Green Pastures Events   0.142542
9   8l0zeJCRfAL8BYrnDxPcJQ                       Fonda San Miguel   0.140678
10  GfO55IFjBnA1XXm6JH8Sbg                      Tuk Tuk Thai Cafe   0

In [18]:
item_pivot_matrix_means=item_pivot_matrix.mean(axis=1)
user_pivot_matrix_means = user_pivot_matrix.mean(axis=1)
item_pivot_matrix_entiremean = np.nanmean(item_pivot_matrix)

pred_rating=0
sim_numer=[]
sim_denom=[]
mu = item_pivot_matrix_entiremean
b_x = user_pivot_matrix_means[user_user_input] - mu 
b_i = item_pivot_matrix_means[item_user_input]  -  mu
b_xi = mu + b_x + b_i

#Calculate predicted rating for each recommended restaurant
for x in range(len(item_top_businesses)): 
    for y in range(len(reviews)):
        
        #checking to see if there are other restaurants reviewed by same user 
        #if it is, then calculate predicted rating
        if bool(reviews[y]['business_id']==item_top_businesses[x]) & bool(reviews[y]['user_id']==user_user_input):
            sim_numer.append(item_top_correlations[x]*(reviews[y]['stars'] - (mu + b_x + item_pivot_matrix_means[item_top_businesses[x]]-mu)))
            sim_denom.append(item_top_correlations[x])
            pred_rating = b_xi + (sum(sim_numer)/sum(sim_denom))



print("Predicted Rating for Restaurant: ", item_user_input,"is ", pred_rating)

Predicted Rating for Restaurant:  hpAOH8cmY5WHjnl8jGQ1dA is  4.465271689108746


In [19]:
for x in range(len(item_top_businesses)): 
    for y in range(len(reviews)):
        
        #checking to see if there are other restaurants reviewed by same user 
        #if it is, then calculate predicted rating
        if bool(reviews[y]['business_id']==item_top_businesses[x]) & bool(reviews[y]['user_id']==user_user_input):
            print(reviews[y]['business_id'])

M-kdNZD-RhloIH5NZtS59A
Xuyfcldi878hjqQPAOtlCA
1Eq16r_MSnvPsnIykBdy9w
NtiDiLoIOaDwzWGabJ0qsg


# CF Item-item, predicted ratings

In [20]:
item_pivot_matrix_means=item_pivot_matrix.mean(axis=1)
user_pivot_matrix_means = user_pivot_matrix.mean(axis=1)
item_pivot_matrix_entiremean = np.nanmean(item_pivot_matrix)

pred_rating=0
sim_numer=[]
sim_denom=[]
mu = item_pivot_matrix_entiremean
b_x = user_pivot_matrix_means[user_user_input] - mu 
b_i = item_pivot_matrix_means[item_user_input]  -  mu
b_xi = mu + b_x + b_i

#Calculate predicted rating for each recommended restaurant
for x in range(len(item_top_businesses)): 
    for y in range(len(reviews)):
        
        #checking to see if there are other restaurants reviewed by same user 
        #if it is, then calculate predicted rating
        if bool(reviews[y]['business_id']==item_top_businesses[x]) & bool(reviews[y]['user_id']==user_user_input):
            sim_numer.append(item_top_correlations[x]*(reviews[y]['stars'] - (mu + b_x + item_pivot_matrix_means[item_top_businesses[x]]-mu)))
            sim_denom.append(item_top_correlations[x])
            pred_rating = b_xi + (sum(sim_numer)/sum(sim_denom))



print("Predicted Rating for Restaurant: ", item_user_input,"is ", pred_rating)

Predicted Rating for Restaurant:  hpAOH8cmY5WHjnl8jGQ1dA is  4.465271689108746


In [21]:
for x in range(len(item_top_businesses)): 
    for y in range(len(reviews)):
        
        #checking to see if there are other restaurants reviewed by same user 
        #if it is, then calculate predicted rating
        if bool(reviews[y]['business_id']==item_top_businesses[x]) & bool(reviews[y]['user_id']==user_user_input):
            print(reviews[y]['business_id'])

M-kdNZD-RhloIH5NZtS59A
Xuyfcldi878hjqQPAOtlCA
1Eq16r_MSnvPsnIykBdy9w
NtiDiLoIOaDwzWGabJ0qsg
